In [1]:
import bw2io
from pathlib import Path
import bw2data
from brightpath import SimaproConverter
# Simapro CSV file
fp="Strathclyde db/strathclyde_space_systems_database_1_0_3.csv"
bw2data.projects.set_current("ecoinvent-3.9.1-cutoff")
c = SimaproConverter(fp, ecoinvent_version="3.9")

Extracted 992 unallocated datasets in 0.51 seconds
Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: assign_only_product_as_production
Applying strategy: drop_unspecified_subcategories
Applying strategy: sp_allocate_products
Applying strategy: fix_zero_allocation_products
Applying strategy: split_simapro_name_geo
Applying strategy: strip_biosphere_exc_locations
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: set_code_by_activity_hash
Applying strategy: link_technosphere_based_on_name_unit_location
Applying strategy: change_electricity_unit_mj_to_kwh
Applying strategy: set_lognormal_loc_value_uncertainty_safe
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_simapro_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: normalize_simapro_biosphere_names
Applying strategy: migrate_exchanges
Applying strategy: fix_localized_water_flows
Ap

In [2]:
# brightpath function
c.convert_to_brightway()

- format exchanges
- remove empty datasets
- remove empty exchanges
- check inventories
Done!


In [3]:
# apply migration file previosuly generated
c.i.data = bw2io.strategies.migrate_exchanges(
   db=c.i.data,
   migration="SSSD-ei-3.9_migration"
)

In [4]:
c.i.match_database(fields=["name", "reference product", "location"])
c.i.match_database("ecoinvent-3.9.1-cutoff", fields=["name", "reference product", "location"])
c.i.match_database("ecoinvent-3.9.1-biosphere", fields=["name", "categories"])

Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields


In [5]:
# unlinked exchanges are fine to be dropped
# must revise the migration file otherwise
c.i.statistics()

1048 datasets
8494 exchanges
803 unlinked exchanges
  Type biosphere: 259 unique unlinked exchanges
  Type technosphere: 4 unique unlinked exchanges


(1048, 8494, 803)

In [7]:
c.i.drop_unlinked(i_am_reckless=True)
if c.i.db_name in bw2data.databases:
    del bw2data.databases[c.i.db_name]
#c.i.write_database()

Applying strategy: drop_unlinked
Applied 1 strategies in 0.01 seconds


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 01/29/2025 17:35:24
  Finished: 01/29/2025 17:35:24
  Total time elapsed: 00:00:00
  CPU %: 90.10
  Memory %: 0.82
Created database: strathclyde_space_systems_database_1_0_3


Brightway2 SQLiteBackend: strathclyde_space_systems_database_1_0_3

In [8]:
# export back to Brightway's Excel format
bw2io.export.write_lci_excel("strathclyde_space_systems_database_1_0_3")

'/Users/romain/Library/Application Support/Brightway3/ecoinvent-391-cutoff.fa1f817c134dcb954f38875dca138721/output/lci-strathclyde_space_systems_database_1_0_3.xlsx'

In [5]:
# this is to build a migration file from teh previous import attempt
migration = {
    "fields": ["name", "reference product", "location", ],
}

migration["data"] = [
    (
        (u["name"], u["reference product"], u["location"], ),
        {}
    )
    for u in c.i.unlinked
    if u["type"] == "technosphere"
]

In [6]:
# save migration file to json
import json
with open("SSSD-ei-3.9_migration.json", 'w') as fp:
    json.dump(migration, fp, indent=2)

In [9]:
# load it back after populating it
import json
with open("SSSD-ei-3.9_migration.json") as f_in:
    migration = json.load(f_in)

In [10]:
# register to migration file
bw2io.Migration(name="SSSD-ei-3.9_migration").write(data=migration, description="SSSD-ei-3.9_migration")

In [5]:
# we try to import now the Excel export for ei 3.9 
# to link it to ei 3.10
import bw2io
from pathlib import Path
import bw2data
from brightpath import SimaproConverter
fp="lci-SSSD_1.0.3_ei391.xlsx"
bw2data.projects.set_current("ecoinvent-3.10-cutoff")

In [6]:
i = bw2io.ExcelImporter(fp)

Extracted 1 worksheets in 0.62 seconds


In [7]:
i.apply_strategies()

Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 3.19 seconds


In [8]:
i.match_database(fields=["name", "reference product", "location"])
i.match_database("ecoinvent-3.10-cutoff", fields=["name", "reference product", "location"])
i.match_database("ecoinvent-3.10-biosphere", fields=["name", "categories"])

Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields


In [9]:
i.statistics()

1047 datasets
7690 exchanges
173 unlinked exchanges
  Type biosphere: 145 unique unlinked exchanges
  Type technosphere: 1 unique unlinked exchanges


(1047, 7690, 173)

In [10]:
# onle one technosphere exchange unlink
[u for u in i.unlinked if u["type"] == "technosphere"]

[{'name': 'market for sodium hydroxide, without water, in 50% solution state',
  'amount': 0.698,
  'location': 'GLO',
  'unit': 'kilogram',
  'categories': ('Materials/fuels',),
  'type': 'technosphere',
  'uncertainty type': 0,
  'loc': 0.698,
  'comment': 'Material Input: Sodium hydroxide',
  'product': 'sodium hydroxide, 50% in H2O, production mix, at plant',
  'reference product': 'sodium hydroxide, without water, in 50% solution state',
  'simapro name': 'sodium hydroxide, 50% in H2O, production mix, at plant {RER}'}]

In [11]:
# we use a migration file generated by bw_ecoinvent_migrations
import json
with open("ecoinvent-3.9.1-cutoff-ecoinvent-3.10-cutoff.json") as f_in:
    migration = json.load(f_in)

In [12]:
migration = {
    (
        a["source"]["name"],
        a["source"]["reference product"],
        a["source"]["location"],
        #a["source"]["unit"]
    ): a["targets"]
    for a in migration["disaggregate"]
}
len(migration)

183

In [13]:
for act in i.data:
    new_excs = []
    for exc in act["exchanges"]:
        if exc["type"] == "technosphere":
            key = (
                exc["name"],
                exc["reference product"],
                exc["location"],
                #exc["unit"]
            )
            if key in migration:
                for target in migration[key]:
                    new_excs.append(
                        {
                            "name": target["name"],
                            "reference product": target["reference product"],
                            "unit": exc["unit"],
                            "location": target["location"],
                            "uncertainty type": exc.get("uncertainty type", 0),
                            "loc": exc.get("loc", 0),
                            "scale": exc.get("scale", 0),
                            "amount": exc["amount"] * target["allocation"],
                            "type": "technosphere"
                        }
                    )
                exc["delete"] = True
    if len(new_excs) > 0:
        act["exchanges"].extend(new_excs)
    act["exchanges"] = [e for e in act["exchanges"] if e.get("delete", False) is False]
                

In [14]:
# no more unlinked technosphere exchanges ≈
i.match_database("ecoinvent-3.10-cutoff", fields=["name", "reference product", "location"])
i.statistics()

Applying strategy: link_iterable_by_fields
1047 datasets
7694 exchanges
169 unlinked exchanges
  Type biosphere: 145 unique unlinked exchanges


(1047, 7694, 169)

In [15]:
# we load a migration file for biosphere flows
import json
with open("ecoinvent-3.9.1-biosphere-ecoinvent-3.10-biosphere.json") as f_in:
    migration = json.load(f_in)

In [16]:
migration = {
    (
        a["source"]["name"],
        #a["source"]["reference product"],
        #a["source"]["location"],
        #a["source"]["unit"]
    ): a["target"]
    for a in migration["replace"]
}
len(migration)

486

In [17]:
for act in i.data:
    for exc in act["exchanges"]:
        if exc["type"] == "biosphere":
            key = (exc["name"],)
            if key in migration:
                if "name" in migration[key]:
                    exc["name"] = migration[key]["name"]
                exc["input"] = ("ecoinvent-3.10-biosphere", migration[key]["uuid"])

In [18]:
i.match_database("ecoinvent-3.10-biosphere", fields=["name", "categories"])
i.statistics()

Applying strategy: link_iterable_by_fields
1047 datasets
7694 exchanges
0 unlinked exchanges
  


(1047, 7694, 0)

In [19]:
i.write_database()

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 01/30/2025 10:46:04
  Finished: 01/30/2025 10:46:04
  Total time elapsed: 00:00:00
  CPU %: 91.70
  Memory %: 1.26
Created database: strathclyde_space_systems_database_1_0_3


In [20]:
bw2io.export.write_lci_excel("strathclyde_space_systems_database_1_0_3")

'/Users/romain/Library/Application Support/Brightway3/ecoinvent-310-cutoff.83f3565fcb85bf5a57f44f5f1e703837/output/lci-strathclyde_space_systems_database_1_0_3.xlsx'

In [13]:
import bw2io
from pathlib import Path
import bw2data
from brightpath import SimaproConverter
fp="lci-SSSD_1_0_3_ei310.xlsx"
bw2data.projects.set_current("ecoinvent-3.11-cutoff")

In [14]:
i = bw2io.ExcelImporter(fp)
i.apply_strategies()
i.match_database(fields=["name", "reference product", "location"])
i.match_database("ecoinvent-3.11-cutoff", fields=["name", "reference product", "location"])
i.match_database("biosphere3", fields=["name", "categories"])
i.statistics()

Extracted 1 worksheets in 0.63 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 0.30 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Graph statistics for `strathclyde_space_syst

(1047, 7694, 5, 0)

In [15]:
import json
with open("ecoinvent-3.10.1-biosphere-ecoinvent-3.11-biosphere.json") as f_in:
    migration = json.load(f_in)

In [16]:
migration = {
    (
        a["source"]["name"],
        #a["source"]["reference product"],
        #a["source"]["location"],
        #a["source"]["unit"]
    ): a["target"]
    for a in migration["replace"]
}
len(migration)

12

In [17]:
for act in i.data:
    for exc in act["exchanges"]:
        if exc["type"] == "biosphere":
            key = (exc["name"],)
            if key in migration:
                if "name" in migration[key]:
                    print("yes")
                    exc["name"] = migration[key]["name"]
                exc["input"] = ("biosphere3", migration[key]["uuid"])

yes
yes
yes
yes
yes


In [18]:
i.match_database("biosphere3", fields=["name", "categories"])
i.statistics()

Applying strategy: link_iterable_by_fields
Graph statistics for `strathclyde_space_systems_database_1_0_3` importer:
1047 graph nodes:
	process: 1047
7694 graph edges:
	technosphere: 5023
	biosphere: 1624
	production: 1047
7694 edges to the following databases:
	strathclyde_space_systems_database_1_0_3: 6053
	biosphere3: 1624
	ecoinvent-3.11-cutoff: 17
0 unique unlinked edges (0 total):




(1047, 7694, 0, 0)

In [19]:
if i.db_name in bw2data.databases:
    del bw2data.databases[i.db_name]
i.write_database()

13:04:29 [warning  ] Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|█████████████████████████████████████| 1047/1047 [00:00<00:00, 1522.21it/s]

13:04:31 [info     ] Vacuuming database            


Created database: strathclyde_space_systems_database_1_0_3


In [20]:
bw2io.export.write_lci_excel("strathclyde_space_systems_database_1_0_3")

'/Users/romain/Library/Application Support/Brightway3/ecoinvent-311-cutoff.48ac3d3e3f6866ef5f407f0df49b277a/output/lci-strathclyde_space_systems_database_1_0_3.xlsx'